# GOAL

Find Majority Vote - AMONG multiple decisions trees.



# Steps

## Pre-testing

Given the whole dataset:
- Get a subset of the dataset
- Create a decision tree
- Repeat for as many times as the number of trees

For each **Decision Tree...**

### What needs to be decided on?

- Split feature
- Split point
- When to stop splitting

1. Calculate **information gain (IG)** with each possible split
2. Divide set with that feature and value that gives the most IG
3. Divide tree and do the same for all created branches...
4. Repeat Steps 1-3 until a **stopping criteria** is reached.


## Testing

### Given a data point:
- Follow the tree until you reach a leaf node.

    - If the leaf is PURE, we return the class of the leaf node. 
    - If the leaf is NOT PURE, we return the most common class label **(Majority Vote Method)**

### Finally

- Get the predictions from each tree
- *Classification:* hold a majority vote
- *Regression:* get the mean of the predictions

In [1]:
#Run Decision Tree Code

import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None,
                 threshold=None, left = None, 
                 right = None, *, value = None):
        self.feature = feature  
        self.threshold = threshold 
        self.left = left 
        self.right = right 
        self.value= value
    
    def is_leaf_node(self):
        return self.value is not None 
    
class DecisionTree:
    def __init__(self, min_samples_split = 2, max_depth = 100, n_features = None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None
        
        
    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)   
        self.root = self._grow_tree(X, y) 
        
    def _grow_tree(self, X, y, depth = 0 ):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))
        
        #check the stopping criteria
        if (depth >= self.max_depth or n_labels == 1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        feat_idx = np.random.choice(n_feats, self.n_features, replace = False) 
        
        #find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idx) 
        
        # CODE PATH
        #Pass the arguement(X, y, feat_idx)  and load _best_split
            #Best split loads and calls information gain
                #information gain loads and calls _entropy and _split
                    #_entropy returns entropy value
                    #_split returns left_idxs, right_idxs
            #backtrack to information gain and return  information_gain = parent_entropy - child_entropy
            
        #Backtrack to _best_split and the IG is stored in gain
            #if gain > best_gain (which starts at -1)
                #updated best_gain with gain
                #split_idx = feat_idx
                #split_threshold = thr
        #best_feature = split_idx, best_thresh = split_threshold
        
        #From there create the Child Nodes
        
        #create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree( X[left_idxs, :], y[left_idxs], depth+1) #Increase depth by 1 OR current depth + 1
        right = self._grow_tree( X[right_idxs, :], y[right_idxs], depth+1) #Increase depth by 1 OR current depth + 1
        return Node(best_feature, best_thresh, left, right)
        
    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None
        
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds =  np.unique(X_column)
            
            for thr in thresholds: 
                gain = self._information_gain(y, X_column, thr)
                
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr
                    
        return split_idx, split_threshold
    
    #Main calculation for IG
    
    def _information_gain(self, y, X_column, threshold):
        #parent entropy
        parent_entropy = self._entropy(y)
        
        #create children
        left_idxs, right_idxs = self._split(X_column, threshold)
        
        if len(left_idxs) == 0 or len(right_idxs) == 0: #if the left OR right indices is empty
            return 0 
        
        #calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs) #no. of samples in left, no. of samples in right
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs]) #entropy of left, entropy of right
        
        #child entropy is (no. of samples /total no.of samples) * (entropy of children) THEREFORE
        child_entropy = ( (n_l/n) * e_l ) + ( (n_r/n) * e_r )
        
        #calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain
        
    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs
    
    def _entropy(self, y):
        hist = np.bincount(y)
        p_x = hist/len(y) #p(X)
        
        return -np.sum([p*np.log(p) for p in p_x if p>0]) #Again, why doesn't she add the 2 for log2?
    
    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value
    
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
        
    def _traverse_tree(self, x, node):
        if node.is_leaf_node(): #I'm assuming class object name is cases insensitive so Node.is_leaf or node. is_leaf
            return node.value
        
        #left, right contains a certain "feature we divided it with" 
        #X[left_idxs, :]
        
        #x[node.feature] = feature we divided it with
        
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left) #if less, we traverse on the left
        return self._traverse_tree(x, node.right) #else, we traverse right

In [2]:
# Intial Code

In [3]:
import numpy as no
from collections import Counter

class RandomForest:
    def __init__(self, n_trees=10, max_depth =10, min_samples_split = 2, n_features = None): #initial values chosen at random
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []
        
    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            
            tree = DecisionTree(max_depth = self.max_depth,
                        min_samples_split = self.min_samples_split,
                        n_features = self.n_features)
            
            X_sample, y_sample = self._bootstrap_samples(X,y) #Helper function 1
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)
            
    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0] #In a numpy array, first index is number of samples
                                # second index is number of features
            
        idxs = np.random.choice(n_samples, n_samples, replace = True) #After a sample is selected in the dataset
                                                                      #some samples would be dropped, some samples will be used AGAIN
        return X[idxs], y[idxs]
    
    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0] #Gets the most common occurence of a certain value from a reconstructed array.
        return most_common
        
    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        
            #result of predictions -- "a list of lists"
            # One list per tree --> [ [1,0,1,1],[0,0,1,1],[] ]
                #First Tree: 1 is the first sample, 0 second sample etc.
            #" A list of lists" --> [[1,0]]
                #1 is the prediction for the 1st tree, 0 is the prediction of the 2nd tree
            
        tree_preds = np.swapaxes(predictions, 0, 1)
        predictions = np.array([self._most_common_label(pred) for pred in tree_preds])
        return predictions
    

# Load Dataset

In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
#from RandomForest import RandomForest

data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 1234
)

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) /  len(y_true)
    return accuracy

clf = RandomForest()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

acc = accuracy(y_test, predictions)
print(acc)

0.9035087719298246


# Change values

In [5]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
#from RandomForest import RandomForest

data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 1234
)

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) /  len(y_true)
    return accuracy

clf = RandomForest(n_trees = 20)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

acc = accuracy(y_test, predictions)
print(acc)

0.9122807017543859


In [6]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
#from RandomForest import RandomForest

data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 1234
)

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) /  len(y_true)
    return accuracy

clf = RandomForest(n_trees = 20, max_depth = 5)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

acc = accuracy(y_test, predictions)
print(acc)

0.9210526315789473
